<a href="https://colab.research.google.com/github/bindupriya0309/AMAZON-CLONE/blob/main/Task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install -U langchain-community

In [5]:
!pip install fitz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 71.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.6/95.6 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.5/421.5 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 528.1/528.1 kB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 76.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.8 MB/s eta 0:00:00


In [6]:
!pip install pytesseract

In [7]:
import os
import pickle
import time
import requests
from bs4 import BeautifulSoup
from langchain_groq import ChatGroq
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA

# Initialize LLM
llm = ChatGroq(temperature=0, groq_api_key="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", model_name="llama-3.1-70b-versatile")

file_path = "faiss_store_openai.pkl"

# Function to scrape content from a website
def scrape_website(url):
    try:
        # Send a request to the website
        response = requests.get(url)

        if response.status_code == 200:
            # Parse the HTML content of the page
            soup = BeautifulSoup(response.content, 'html.parser')
            # Extract text from all paragraph tags (you can adjust this as needed)
            paragraphs = soup.find_all('p')
            text = ' '.join([para.get_text() for para in paragraphs])
            return text
        else:
            print(f"Failed to retrieve the webpage: {url} with status code {response.status_code}")
            return ""
    except Exception as e:
        print(f"Error scraping {url}: {e}")
        return ""

# Process websites after input
def process_websites(urls):
    all_text = ""

    # Scrape text from all websites
    for url in urls:
        print(f"Processing website: {url}")
        extracted_text = scrape_website(url)
        all_text += extracted_text + "\n"

    if not all_text:
        print("No text was extracted from the websites.")
        return

    # Split text into chunks
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    text_chunks = text_splitter.split_text(all_text)

    # Create embeddings and vector store
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vectorstore_openai = FAISS.from_texts(text_chunks, embeddings)

    # Save FAISS index
    print("Embedding Vector Started Building...✅✅✅")
    time.sleep(2)

    # Save the FAISS index to a pickle file
    try:
        with open(file_path, "wb") as f:
            pickle.dump(vectorstore_openai, f)
        print("Text extracted and FAISS index saved.")
    except Exception as e:
        print(f"Error saving FAISS index: {e}")

# User input for URLs
urls_input = input("Enter the website URLs (comma-separated): ")
urls = [url.strip() for url in urls_input.split(',')]

if not urls:
    print("No URLs provided.")
else:
    # Run processing after URL input
    process_websites(urls)

    # Query input
    query = input("Ask a Question: ")
    if query:
        if os.path.exists(file_path):
            try:
                with open(file_path, "rb") as f:
                    vectorstore = pickle.load(f)
                    chain = RetrievalQA.from_llm(llm=llm, retriever=vectorstore.as_retriever())

                # Get response
                result = chain.run(query)

                # Display answer
                print("Answer:")
                print(result)
            except Exception as e:
                print(f"Error processing query: {e}")
        else:
            print(f"FAISS index file not found at {file_path}. Please ensure you run the scraping process first.")


Processing website: https://www.washington.edu/
Embedding Vector Started Building...✅✅✅
Text extracted and FAISS index saved.
Ask a Question: academics
Answer:
Based on the provided context, here are some academic-related points mentioned:

1. UW astronomy undergrads are using coding skills to help scientists with discoveries from a new telescope.
2. Chris Mantegna, '21, is studying how pollutants affect shellfish in the food web.
3. David Baker, a professor of biochemistry at the UW School of Medicine, received the 2024 Nobel Prize in Chemistry for computational protein design.
4. Nocona Abrams (B.A. '19, M.S.W. '23) is a therapist who returned to Washington's Yakima Valley to improve mental health care.
5. Dr. Ezzeddine has joined the University of Washington, bringing a global engagement perspective and understanding of higher education.

These points highlight various academic achievements and pursuits within the University of Washington community.
